In [1]:
DBPATH = './28_test.db'

import time
import numpy as np

import qcodes as qc
from qcodes import load_or_create_experiment, Measurement, Parameter

qc.config.core.db_location = DBPATH
qc.initialise_database()

from pytopo.sweep import sweep

In [ ]:
x = Parameter('x')
y = Parameter('y')
z = Parameter('z')
data = Parameter('data')

station = qc.Station(x, y, z, data)

In [ ]:
xvals = np.arange(5).astype(float)
yvals = np.arange(5).astype(float)
zvals = np.arange(5).astype(float)
xx, yy, zz = np.meshgrid(xvals, yvals, zvals, indexing='ij')
dd = xx + yy + zz

def soft_3d_sweep():
    for x, y, z, data in zip(xx.reshape(-1), yy.reshape(-1), zz.reshape(-1), dd.reshape(-1)):
        yield x, y, z, data

exp = load_or_create_experiment('soft_3d_sweep_test', sample_name='no sample')

meas = Measurement(exp, station)
meas.register_parameter(x)
meas.register_parameter(y)
meas.register_parameter(z)
meas.register_parameter(data, setpoints=(x, y, z))
meas.write_period = 2

with meas.run() as datasaver:
    for xval, yval, zval, dataval in soft_3d_sweep():
        print(f"x = {xval}, y = {yval}, z={zval} {10*' '}", end='\r')
        datasaver.add_result(
            (x, xval),
            (y, yval),
            (z, zval),
            (data, dataval),
        )
        time.sleep(0.5)

In [5]:
from plottr.data.qcodes_dataset import datadict_from_path_and_run_id
from plottr.data.datadict import datadict_to_meshgrid

In [6]:
data = datadict_from_path_and_run_id(DBPATH, run_id=4)
data = datadict_to_meshgrid(data)

In [7]:
data

{'data': {'unit': '',
  'axes': ['x', 'y', 'z'],
  'values': array([[[0, 1, 2, 3, 4],
          [1, 2, 3, 4, 5],
          [2, 3, 4, 5, 6],
          [3, 4, 5, 6, 7],
          [4, 5, 6, 7, 8]],
  
         [[1, 2, 3, 4, 5],
          [2, 3, 4, 5, 6],
          [3, 4, 5, 6, 7],
          [4, 5, 6, 7, 8],
          [5, 6, 7, 8, 9]],
  
         [[2, 3, 4, 5, 6],
          [None, None, None, None, None],
          [None, None, None, None, None],
          [None, None, None, None, None],
          [None, None, None, None, None]]], dtype=object)},
 'x': {'unit': '', 'axes': [], 'values': array([[[0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]],
  
         [[1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1]],
  
         [[2, 2, 2, 2, 2],
          [None, None, None, None, None],
          [None, None, None, None, None],
          [None, None,